In [1]:
import numpy as np
import matplotlib.pyplot as plt

# import pyomo.environ as pyo
# from pyomo.environ import ConcreteModel, Var, ConstraintList, Objective, SolverFactory, value, RangeSet, Constraint, Param
# from pyomo.dae import *

import jax
import jax.numpy as jnp

from scipy.interpolate import interp1d

import sys
import os

collocation2_path = os.path.abspath(os.path.join('..', 'utils'))

# Add the directory to sys.path
if collocation2_path not in sys.path:
    sys.path.append(collocation2_path)

from collocation import compute_weights, lagrange_derivative
from interpolation import BarycentricInterpolation
from data_generation import generate_ode_data
from non_parametric_collocation import collocate_data
from optimization_pyomo import optimization_model

In [2]:
#---------------------------------------------DATA PARAMS---------------------------------------------#
N = 200
noise_level = 0.5
ode_type, params = "harmonic_oscillator", {"omega_squared": 1}
start_time, end_time = 0, 10
spacing_type = "chebyshev"
initial_state = jnp.array([0.0, 1.0])

#--------------------------------------------GENERATE DATA--------------------------------------------#
t, y, y_noisy, true_derivatives = generate_ode_data(N, noise_level, ode_type, params, start_time, end_time, spacing_type, initial_state)

# numpy array is required for pyomo
y_noisy = np.array(jnp.squeeze(y_noisy))
t = np.array(jnp.squeeze(t))

#---------------------------COLLOCATION MATRIX PREPARATION--------------------------#
weights = compute_weights(t)
D = np.array(lagrange_derivative(t, weights))

In [3]:
print(optimization_model.__doc__)


    Constructs and solves an optimization model using neural networks to fit observed data points to an ODE system with collocation constraints.

    This function sets up a neural network within a Pyomo optimization framework to learn dynamics from observed data points, 
    minimizing the difference between observed states and predicted states by adjusting the neural network's weights and biases 
    subject to ODE constraints defined via a collocation matrix.

    Parameters:
    y_observed (np.ndarray): An array of observed data points with shape (N, 2), where N is the number of time points and 2 corresponds to the state variables u and v.
    t (np.ndarray): An array of time points corresponding to the observations in y_observed.
    first_derivative_matrix (np.ndarray): A matrix representing the first derivative constraints imposed by a collocation method on the ODE solution.
    penalty_lambda (float, optional): A regularization parameter used to scale the penalty terms in the 

In [ ]:
model = optimization_model(y_noisy, max_iter=500)